In [ ]:
from os import listdir
from os.path import isfile, join
import fnmatch
import pandas as pd


# *************** change only model name in the following line ******************** 
#model_path = r"csv_files/decomp_dp_age30_1/test/"
#model_path_val = r"csv_files/dp_black_1_test/val/"
model_path = r"csv_files/original/test/"

# take the csv files in that directory
files = [f for f in listdir(model_path) if fnmatch.fnmatch(f, '*test_isotonic*.csv')]
#files = [f for f in listdir(model_path) if fnmatch.fnmatch(f, '*.csv')]


print(files)

In [ ]:
# add age, race information into the csv files

for f in files:
    analysis2 = pd.read_csv(model_path+f)
    print(analysis2.shape)

    output_result = 'temp_' + f

    ###############################################
    # spliting admission id values
    ##############################################

    analysis2[['id','not_needed','not_needed2']] = analysis2.stay.str.split('_',expand=True,)
    analysis2

    ################################################
    # delete the "not_needed" and "not_needed2" part
    ################################################
    analysis2.drop(["not_needed", "not_needed2"],axis=1, inplace=True)
    analysis2

    analysis2.rename(columns={'id':'SUBJECT_ID'}, inplace= True)

    admi = pd.read_csv(r'D:\Google Drive\PhD\Dr. Daphne Yao\Mimic Project\JupyterNotebookFiles_new\MIMIC\decomp\new\Experiment 2\ADMISSIONS.csv')

    admi_modified = admi[['SUBJECT_ID','ETHNICITY','ADMITTIME']]
    admi_modified.drop_duplicates(subset=["SUBJECT_ID"],keep="first" , inplace=True)
    analysis2 = analysis2.astype({'SUBJECT_ID':'int64'})
    result = pd.merge(analysis2,admi_modified,how='left', on='SUBJECT_ID')

    patients = pd.read_csv(r'D:\Google Drive\PhD\Dr. Daphne Yao\Mimic Project\JupyterNotebookFiles_new\MIMIC\decomp\new\Experiment 2\PATIENTS.csv')
    patients_modified = patients[['SUBJECT_ID','GENDER','DOB']]
    patients_modified.drop_duplicates(subset=["SUBJECT_ID"],keep="first", inplace=True)

    result2 = pd.merge(result,patients_modified,how='left', on='SUBJECT_ID')
    result2['DOB'] = pd.to_datetime(result2.DOB)
    result2['ADMITTIME'] = pd.to_datetime(result2.ADMITTIME)
    result2['AGE'] = result2['ADMITTIME'].dt.year - result2['DOB'].dt.year
    #drop unnecessary column
    result2 = result2.drop(columns = ['DOB', 'ADMITTIME', 'SUBJECT_ID'])

    result2.to_csv(model_path+output_result, index=False)


